## Lang Chain 
Esta seção mostra como usar embeddings para busca semântica e responder a perguntas sobre bases de dados.
Ela usa a biblioteca LangChain.

In [1]:
from dotenv import load_dotenv
load_dotenv()
import openai
import re
import requests
import sys
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = "2022-12-01"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT
COMPLETIONS_MODEL = os.getenv('DEPLOYMENT_NAME')



In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA

### Indexando documentos em uma base vetorial


In [4]:
from langchain.document_loaders import TextLoader
loader = TextLoader("./data/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(chunk_size=1)
docsearch = FAISS.from_documents(texts, embeddings)
print('done')

done


### Buscando no documento


In [5]:
qa = RetrievalQA.from_chain_type(llm=AzureOpenAI(temperature=0, deployment_name="text-davinci-003", model_name="text-davinci-003"), chain_type="stuff", retriever=docsearch.as_retriever())

query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."

### Salvando o arquivo e carregando de novo


In [15]:
docsearch.save_local('./data/state_union/')


loaded = FAISS.load_local('./data/state_union/', embeddings)
